In [1]:
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import plotly.express as px

In [2]:
# Get current date, time and timezone to print to the App
now = datetime.datetime.now()
datetime_now = now.strftime("%d/%m/%Y %H:%M:%S")
timezone_string = datetime.datetime.now().astimezone().tzname()

In [3]:
# read csv files
articles_data = pd.read_csv("./datasets/NIFTY_500_Articles.csv", index_col=0)
ticker_metadata = pd.read_csv("./datasets/ticker_metadata.csv", index_col=0)

In [4]:
# Universe filter variable
universe = "nifty_50"

# universe string will be used at places where we need to show the universe name (without underscores)
if universe == "nifty_50":
    universe_string = "NIFTY 50"
if universe == "nifty_100":
    universe_string = "NIFTY 100"
if universe == "nifty_200":
    universe_string = "NIFTY 200"
if universe == "nifty_500":
    universe_string = "NIFTY 500"

universe_tickers = pd.read_csv("./datasets/{}.csv".format(universe), index_col=0)[
    "Symbol"
]

In [6]:
# date and time filter variables
date_interval = 30
cutoff_date = pd.to_datetime(
    datetime.datetime.now() - relativedelta(days=date_interval)
)

In [7]:
filterd_articles = articles_data.loc[
    articles_data.Date.astype("datetime64[ns]") > cutoff_date
]

In [14]:
filterd_articles

,Ticker,Headline,Date,Source,Link,positive,neutral,negative,compound
0,ADANIENT,Adani Enterprises to launch first-ever public ...,2024-07-30 11:35:41,Business Standard,https://www.business-standard.com/companies/ne...,0.699481,0.292175,0.008345,0.691136
1,ADANIENT,Adani Total Gas Q1 Results: PAT jumps 15% YoY ...,2024-07-29 18:35:41,The Economic Times,https://m.economictimes.com/markets/stocks/ear...,0.948632,0.026765,0.024603,0.924030
2,ADANIENT,Adani Wilmar Jumps Over 5% As Company Swings T...,2024-07-30 02:35:41,News18,https://www.news18.com/business/markets/adani-...,0.883750,0.055698,0.060552,0.823197
3,ADANIENT,Adani Wilmar Shares Rocket 7% After Posting ₹3...,2024-07-29 18:35:41,Benzinga,https://in.benzinga.com/content/40004312/adani...,0.921151,0.024612,0.054237,0.866915
4,ADANIPORTS,"Stocks to buy: Adani Ports, Cipla, Ashok Leyla...",2024-07-29 18:35:41,mint,https://www.livemint.com/market/stock-market-n...,0.916255,0.068680,0.015064,0.901191
...,...,...,...,...,...,...,...,...,...
188,WIPRO,Wipro Share Price Highlights : Wipro closed to...,2024-07-29 20:36:02,Livemint,https://www.livemint.com/market/live-blog/wipr...,0.007338,0.017221,0.975441,-0.968103
189,WIPRO,"Wipro lags peers, revenue down nearly 5% in Ju...",2024-07-23 18:36:02,Times of India,https://timesofindia.indiatimes.com/business/i...,0.007551,0.017093,0.975355,-0.967804
190,WIPRO,"Q1 results: RIL, Infosys, Paytm, Jio Financial...",2024-07-16 18:36:02,mint,https://www.livemint.com/companies/company-res...,0.102075,0.799176,0.098749,0.003326
191,WIPRO,Wipro vs TCS vs Infosys vs HCL Tech: Which IT ...,2024-07-23 18:36:02,mint,https://www.livemint.com/market/stock-market-n...,0.047133,0.933767,0.019100,0.028033


In [8]:
ticker_scores = (
    filterd_articles.loc[:, ["Ticker", "positive", "negative", "compound"]]
    .groupby("Ticker")
    .mean()
    .reset_index()
)

In [9]:
# filter companies based on universe
filtered_tickers = ticker_scores.loc[ticker_scores.Ticker.isin(universe_tickers)]

In [10]:
final_df = pd.merge(ticker_metadata, filtered_tickers, on="Ticker", how="inner")

In [11]:
final_df.rename(
    columns={"Market Cap": "Market Cap (Billion Rs)", "compound": "Sentiment Score"},
    inplace=True,
)

In [12]:
final_df.head(2)

,Ticker,Sector,Industry,Market Cap (Billion Rs),Company Name,positive,negative,Sentiment Score
0,ADANIENT,Commodities,Metals & Minerals Trading,3521.86,Adani Enterprises Limited,0.863254,0.036934,0.826319
1,ADANIPORTS,Services,Transport Infrastructure,3345.41,Adani Ports and Special Economic Zone Limited,0.653950,0.030728,0.623222


In [13]:
# Plotting
fig = px.treemap(
    final_df,
    path=[px.Constant(universe_string), "Sector", "Industry", "Ticker"],
    values="Market Cap (Billion Rs)",
    color="Sentiment Score",
    hover_data=["Company Name", "negative", "positive", "Sentiment Score"],
    color_continuous_scale=["#FF0000", "#000000", "#00FF00"],
    color_continuous_midpoint=0,
)
# fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(height=800)
fig.update_layout(margin=dict(t=30, l=10, r=10, b=10), font_size=20)